In [23]:
import pandas as pd
import numpy as np
import mne
import re
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchsummary import summary
import torch.autograd as autograd

In [15]:
DATA_PATH = '../data/event_data.csv'
MFF_DIR = '../data/'
STIM_CHANNEL_NAMES = ['201' + str(i) for i in range(10)]
EEG_CHANNEL_NAMES = ['E'+ str(i) for i in range(1,33)]
FILE_ORDER = [
    'ABI-Ashutosh-25-Sept-2019_20190925_102455.mff',
    'ABI-Kartikeya-22-10-19_20191022_033359.mff',
    'ABI-Kartikeya-22-10-19_20191022_040406.mff',
    'ABI-Pranali-23-09-19_20190923_103745.mff',
    'ABI-Shreyas-21-09-19_20190921_033411.mff',
    'ABI2-Ashutosh-25-Sept-2019_20190925_104849.mff',
    'ABI2-Pranali-23-Sept-19_20190923_110343.mff',
    'ABI2-Shreyas-Sept-21_20190921_035826.mff',
    'ABI2_Nishit_16Oct-2019_20191016_111224.mff',
    'ABI_Nishit_16Oct-2019_20191016_104307.mff',  
]
EVENT_LENGTHS = [159] + [200]*9
NUMPY_X_FNAME, NUMPY_Y_FNAME = MFF_DIR + 'X_small.npy',MFF_DIR + 'y_small.npy'

In [ ]:
######### NO NEED TO RUN ###################

In [ ]:
class EEGDataset(Dataset):
    def __init__(self):
        self.df = pd.read_csv(DATA_PATH)
        self.current_file = None
        self.current_raw = None
        self.current_events = None
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        fname = self.df.iloc[idx]['fname']
        if self.current_file != fname or self.current_file is None:
            self.current_file = fname
            self.current_raw = mne.io.read_raw_egi(MFF_DIR + fname,verbose=False,preload=True).pick_channels(
                STIM_CHANNEL_NAMES+EEG_CHANNEL_NAMES)
            self.current_events = mne.find_events(self.current_raw,verbose=False)
        s_time = self.df.iloc[idx]['s_time']/1000.0
        if idx < 159:
            fake_idx = idx
        else:
            fake_idx = (idx - 159)%200 
        epoch = mne.Epochs(
            self.current_raw.copy().pick_types(eeg=True),
            self.current_events[fake_idx].reshape(1,-1),tmin=s_time,tmax=s_time+2.0-1.0/self.current_raw.info['sfreq'],
            baseline=None,verbose=False)
        # TODO: apply filter to eeg data
        X = epoch.get_data()
        y = self.df.iloc[idx]['label']
        
        return X, y

In [ ]:
eeg_dataset = EEGDataset()

In [ ]:
X = []
y = []
i = 0
for xi,yi in eeg_dataset:
    if xi.shape == (1,32,2000):
        print(i)
        X.append(xi)
        y.append(yi)
    i += 1

In [ ]:
X_np = np.concatenate(X, axis=0)
y_np = np.array(y)
print(X_np.shape,y_np.shape)

In [ ]:
np.save(NUMPY_X_FNAME,X_np)
np.save(NUMPY_Y_FNAME,y_np)

In [ ]:
#####################################

In [16]:
X_np = np.load(NUMPY_X_FNAME)
y_np = np.load(NUMPY_Y_FNAME)

In [17]:
X_torch = torch.Tensor(X_np)
y_torch = torch.Tensor(y_np)
dset = TensorDataset(X_torch,y_torch) # create your datset
dloader = DataLoader(dset,shuffle=True,batch_size=32) # create your dataloader

In [18]:
print(X_torch.shape,y_torch.shape)

torch.Size([967, 32, 2000]) torch.Size([967])


In [45]:
# build model
class EEGEncoder(nn.Module):
    def __init__(self, n_i):
        super(EEGEncoder,self).__init__();
        self.n_i = n_i
        
        self.lstm1 = nn.LSTM(n_i, n_i, 1, bidirectional=False)
        self.linear2 = nn.Linear(n_i, n_i)
        self.linear3 = nn.Linear(n_i,n_i)
        self.linear4 = nn.Linear(n_i,10)
        self.softmax = nn.Softmax(dim=2)
        
    def init_hidden(self, batch_size):
        return(autograd.Variable(torch.randn(1, batch_size, self.n_i)),
                   autograd.Variable(torch.randn(1, batch_size, self.n_i)))
        
    def forward(self, x):
        print(x.size(1))
        self.hidden = self.init_hidden(x.size(1))
        
        x = x.reshape(2000,-1,32)
        x, (hn,cn) = self.lstm1(x)
        enc = self.linear2(hn)
        x = F.relu(enc)
        x = self.linear3(x)
        x = self.linear4(x)
        x = self.softmax(x)
        
        return x

In [46]:
# Testing dimensions
eeg_classifier = EEGEncoder(32)
inp,l = torch.randn(2000,2,32),torch.randn(2)
out = eeg_classifier(inp)
print(out.shape, inp.shape)

2
torch.Size([1, 2, 10]) torch.Size([2000, 2, 32])


In [47]:
criterion = nn.CrossEntropyLoss()
optimizer = opt.Adam(eeg_classifier.parameters(),lr=0.001)

In [48]:
losses = []
for epoch in range(5):
    running_loss = 0.0
    
    for i, data in enumerate(dloader,0):
        inp,lab = data
        optimizer.zero_grad()
        
        out = eeg_classifier(inp)
        out = out.reshape(-1,10)
                        
        loss = criterion(out.float(),lab.long())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 1 == 0:
            print('[{}/{}]: loss = {}'.format(epoch+1,i+1,running_loss/50))
            losses.append(running_loss/50)
            running_loss = 0.0
            
print('Finished training')

32
[1/1]: loss = 0.04600159168243408
32
[1/2]: loss = 0.046103897094726565
32
[1/3]: loss = 0.04605252265930176
32
[1/4]: loss = 0.04604339599609375
32
[1/5]: loss = 0.046080403327941895
32
[1/6]: loss = 0.04599217414855957
32
[1/7]: loss = 0.04601802349090576
32
[1/8]: loss = 0.04602993965148926
32
[1/9]: loss = 0.04608822345733642
32
[1/10]: loss = 0.046108460426330565
32
[1/11]: loss = 0.04607489585876465
32
[1/12]: loss = 0.04601302146911621
32
[1/13]: loss = 0.04602396965026855
32
[1/14]: loss = 0.04605020046234131
32
[1/15]: loss = 0.045997314453125
32
[1/16]: loss = 0.04612736701965332
32
[1/17]: loss = 0.04599282741546631
32
[1/18]: loss = 0.04607442378997803
32
[1/19]: loss = 0.04600024223327637
32
[1/20]: loss = 0.046154084205627444
32
[1/21]: loss = 0.046015629768371584
32
[1/22]: loss = 0.04603672981262207
32
[1/23]: loss = 0.04596701622009278
32
[1/24]: loss = 0.04607097625732422
32
[1/25]: loss = 0.045994539260864255
32
[1/26]: loss = 0.04603262424468994
32
[1/27]: loss =

In [50]:
for i, data in enumerate(dloader,0):
        inp,lab = data
        out = eeg_classifier(inp);
        print(out.argmax(dim=2),lab)

32
tensor([[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3]]) tensor([7., 1., 0., 5., 3., 7., 6., 6., 0., 2., 4., 9., 0., 0., 7., 6., 2., 1.,
        8., 5., 8., 5., 2., 7., 7., 9., 0., 4., 0., 0., 8., 5.])
32
tensor([[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3]]) tensor([4., 2., 9., 4., 8., 0., 8., 6., 4., 5., 7., 2., 8., 2., 2., 2., 1., 4.,
        5., 9., 2., 8., 9., 2., 0., 1., 3., 8., 3., 7., 7., 7.])
32
tensor([[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3]]) tensor([5., 4., 2., 4., 8., 3., 0., 5., 6., 6., 3., 9., 2., 4., 7., 3., 2., 9.,
        4., 9., 2., 5., 3., 2., 3., 1., 0., 9., 2., 3., 6., 6.])
32
tensor([[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3]]) tensor([0., 5., 0., 8., 3., 2., 5., 2., 1., 2., 0., 2., 8., 5., 6., 0., 1., 8.,
        3